In [1]:
import chess
import chess.pgn
import numpy as np
import pandas as pd
import os
import pickle
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from collections import Counter

In [2]:
os.chdir('..')

In [3]:
## Which PGN File To Train
max_games = 500000
asset_dir = 'asset'
file_name = '2023_tc_50000_games.pgn'

In [4]:
def load_item_from_file(file_path):
    if os.path.exists(file_path):
        print('loading item from cache...')
        with open(file_path, 'rb') as file:
            items = pickle.load(file)
        print('loaded')
        return items
    else:
        return None

In [5]:
pgns = None
assets_path = os.path.join(os.getcwd(), asset_dir)
single_path = os.path.join(assets_path, file_name)

cached_pgns_file = file_name.split('.')[0] + '_pgn.pkl'
cached_urls_file = file_name.split('.')[0] + '_urls_list.pkl'
cached_ratings_file = file_name.split('.')[0] + '_ratings_list.pkl'
cached_games_file = file_name.split('.')[0] + '_game_arrays.pkl'
cached_pgns_path = os.path.join(assets_path, cached_pgns_file)
cached_urls_path = os.path.join(assets_path, cached_urls_file)
cached_ratings_path = os.path.join(assets_path, cached_ratings_file)
cached_games_path = os.path.join(assets_path, cached_games_file)

chess_games_loaded = True
urls_list = load_item_from_file(cached_urls_path)
ratings_list = load_item_from_file(cached_ratings_path)
game_arrays = load_item_from_file(cached_games_path)

if ratings_list is None:
    print('Creating new ratings_list and urls_list...')
if game_arrays is None:
    print('Creating new game_arrays...')
    chess_games_loaded = False

loading item from cache...
loaded
loading item from cache...
loaded
loading item from cache...
loaded


In [6]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout_rate=0):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, hidden_size)
        self.fc6 = nn.Linear(hidden_size, hidden_size)
        self.fc7 = nn.Linear(hidden_size, hidden_size)
        self.fc_classification = nn.Linear(hidden_size, num_classes)
        self.fc_regression = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 

        out, _ = self.lstm(x, (h0,c0))  
        out = out[:, -1, :]
        
        out = self.dropout(F.relu(self.fc1(out)))
        out = self.dropout(F.relu(self.fc2(out)))
        out = self.dropout(F.relu(self.fc3(out)))
        out = self.dropout(F.relu(self.fc4(out)))
        out = self.dropout(F.relu(self.fc5(out)))
        out = self.dropout(F.relu(self.fc6(out)))
        out = self.dropout(F.relu(self.fc7(out)))
        classification_output = self.fc_classification(out)
        regression_output = self.fc_regression(out)
        return classification_output, regression_output

In [7]:
def combined_loss(classification_output, regression_output, target, alpha=0.5):
    classification_loss = nn.CrossEntropyLoss()(classification_output, target)
    regression_target = target.float()
    regression_loss = nn.MSELoss()(regression_output.squeeze(), regression_target)
    return alpha * classification_loss + (1 - alpha) * regression_loss

def train_model(model, train_loader, test_loader, optimizer, num_epochs, device, alpha=0.5):
    torets = []
    for epoch in range(num_epochs):
        model.train()
        for i, (moves, labels) in enumerate(train_loader):  
            moves = moves.to(device)
            labels = labels.to(device)

            classification_output, regression_output = model(moves)
            loss = combined_loss(classification_output, regression_output, labels, alpha)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        #print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
        predicted_probs, predicted_labels, actual_labels = test_model(model, test_loader, device)
        pred_closeness = [sum(abs(p - a) <= k for p, a in zip(predicted_labels, actual_labels)) for k in range(10)]
        toret = [x/20000 for x in pred_closeness]
        torets.append(toret)
    return torets

def test_model(model, test_loader, device):
    model.eval()
    n_correct = 0
    n_samples = 0
    predicted_probs = []
    predicted_labels = []
    actual_labels = []
    with torch.no_grad():
        for moves, labels in test_loader:
            moves = moves.to(device)
            labels = labels.to(device)
            classification_output, _ = model(moves)
            probabilities = F.softmax(classification_output, dim=1)

            _, predicted = torch.max(classification_output.data, 1)
            predicted_probs.extend(probabilities.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    #print(f'Accuracy of the network on the test moves: {acc} %')
    return predicted_probs, predicted_labels, actual_labels

In [8]:
def pad_game(game, max_length=256, vector_size=42):
    padding_length = max_length - len(game)
    if padding_length < 0:
        return game[:max_length]
    else:
        padding = np.full((padding_length, vector_size), -1)
        return np.vstack((game, padding))

In [9]:
def get_loaders(padded_games, ratings_list, urls_list, batch_size, fold_number=0):
    if fold_number < 0 or fold_number > 4:
        raise ValueError("fold_number must be between 0 and 4")
    test_list = padded_games[fold_number::5]
    #print(len(test_list))
    train_list = [df for i in range(5) if i != fold_number for df in padded_games[i::5]]
    test_ratings = ratings_list[fold_number::5]
    train_ratings = [ratings for i in range(5) if i != fold_number for ratings in ratings_list[i::5]]
    test_urls = urls_list[fold_number::5]
    train_urls = [url for i in range(5) if i != fold_number for url in urls_list[i::5]]

    train_data = [torch.FloatTensor(doc) for doc in train_list]
    test_data = [torch.FloatTensor(doc) for doc in test_list]
    train_labels = torch.LongTensor(train_ratings)
    test_labels = torch.LongTensor(test_ratings)

    train_dataset = TensorDataset(torch.stack(train_data), train_labels)
    test_dataset = TensorDataset(torch.stack(test_data), test_labels)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader, train_urls, test_urls

In [10]:
total_game_counts = [100, 160, 250, 400, 640, 1000, 1600, 2500, 4000, 6400, 10000, 16000, 25000, 40000, 64000, 100000]
sample_arrays = []
sample_ratings = []
sample_urls = []
for multiple in [1,10,100]:
    for sample_size, threshold in zip([2000, 625, 2000, 2000, 625], [2, 1, 5, 8, 4]):
        game_arrays_sample_list = [arr for i, arr in enumerate(game_arrays) if i%sample_size < threshold*multiple]
        ratings_sample_list = [rating for i, rating in enumerate(ratings_list) if i%sample_size < threshold*multiple]
        urls_sample_list = [url for i, url in enumerate(urls_list) if i%sample_size < threshold*multiple]
        sample_arrays.append(game_arrays_sample_list)
        sample_ratings.append(ratings_sample_list)
        sample_urls.append(urls_sample_list)
sample_arrays.append(game_arrays)
sample_ratings.append(ratings_list)
sample_urls.append(urls_list)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [12]:
input_size = 42
hidden_size = 128
num_classes = 10
num_epochs = 21
num_layers = 2
learning_rate = 0.001
dropout_rate = 0.40
sequence_length = 128
batch_size = 100
alpha = 0.8
decay = 0.000010

torch.manual_seed(64)

In [13]:
%%time
for i, game_arrays_list in enumerate(sample_arrays):
    print(f'The model is trained on {total_game_counts[i]} games')
    sample_rating = sample_ratings[i]
    sample_url = sample_urls[i]
    padded_games = [pad_game(g, sequence_length, input_size) for g in game_arrays_list]
    train_loader, test_loader, train_urls, test_urls = get_loaders(padded_games, sample_rating, sample_url, batch_size)

    model_path = file_name.split('.')[0] + '_pred.pth'
    model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=decay)
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'The model has {num_params:,} trainable parameters')
    lists = train_model(model, train_loader, test_loader, optimizer, num_epochs, device, alpha)
    predicted_probs, predicted_labels, actual_labels = test_model(model, test_loader, device)
    pred_closeness = [sum(abs(p - a) <= k for p, a in zip(predicted_labels, actual_labels)) for k in range(10)]
    pred_dist = [5*x/total_game_counts[i] for x in pred_closeness]
    print(pred_dist)
    print()

The model is trained on 100 games
The model has 337,163 trainable parameters
[0.05, 0.3, 0.55, 0.65, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0]

The model is trained on 160 games
The model has 337,163 trainable parameters
[0.0625, 0.25, 0.375, 0.46875, 0.5625, 0.65625, 0.75, 0.84375, 1.0, 1.0]

The model is trained on 250 games
The model has 337,163 trainable parameters
[0.1, 0.32, 0.42, 0.52, 0.58, 0.7, 0.8, 0.92, 1.0, 1.0]

The model is trained on 400 games
The model has 337,163 trainable parameters
[0.125, 0.3, 0.4, 0.5, 0.5875, 0.7125, 0.8, 0.9125, 1.0, 1.0]

The model is trained on 640 games
The model has 337,163 trainable parameters
[0.09375, 0.3125, 0.4921875, 0.59375, 0.71875, 0.8359375, 0.9140625, 1.0, 1.0, 1.0]

The model is trained on 1000 games
The model has 337,163 trainable parameters
[0.115, 0.325, 0.51, 0.605, 0.71, 0.805, 0.925, 1.0, 1.0, 1.0]

The model is trained on 1600 games
The model has 337,163 trainable parameters
[0.1, 0.2875, 0.5375, 0.74375, 0.928125, 0.996875, 1.0, 1.0,

In [14]:
pred_closeness = [sum(abs(p - a) <= k for p, a in zip(predicted_labels, actual_labels)) for k in range(10)]
pred_closeness

[6267, 14358, 18170, 19518, 19891, 19972, 19995, 19999, 20000, 20000]